In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd
import numpy as np
import csv
import ky_utils.ky_baseline_functions as base

Current working directory is now:  /home/jovyan/work/cynthia/interpretable-machine-learning/ky-fl-combined/ky-model


In [2]:
KY_data = pd.read_csv("/home/jovyan/work/cynthia/data/ky-fl-data/KY-cleaned.csv").sort_values('person_id')
FL_data = pd.read_csv("/home/jovyan/work/cynthia/data/ky-fl-data/FL-cleaned.csv").sort_values('person_id')

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['misdemeanor_six_month'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['misdemeanor_six_month'].values

### Model

In [3]:
#### Logistic
c = [1e-4, 1e-3, 0.01, 0.1, 1]
logistic_summary = base.Logistic(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Lasso
c = [1e-4, 1e-3, 0.01, 0.1, 1]
lasso_summary = base.Lasso(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### LinearSVM
c = [1e-4, 1e-3, 0.01, 0.1, 1]
svm_summary = base.LinearSVM(KY_X, KY_Y, FL_X, FL_Y,C=c,seed=816)

#### Random Forest
n_estimators =  [100,150,200]
depth = [7,8,9]
rf_summary = base.RF(KY_X, KY_Y, FL_X, FL_Y,
                     depth=depth, 
                     estimators=n_estimators, 
                     seed=816)

#### XGBoost
learning_rate = [0.1]
depth = [4,5,6]
n_estimators = [100, 150]
xgb_summary = base.XGB(KY_X, KY_Y, FL_X, FL_Y,
                       learning_rate=learning_rate, 
                       depth=depth, 
                       estimators=n_estimators,
                       seed=816)

#### save results
summary_misdemeanor6_ky_model = {"Logistic": logistic_summary,
                                 "Lasso": lasso_summary,
                                 "LinearSVM": svm_summary,
                                 "RF": rf_summary,
                                 "XGBoost": xgb_summary}

/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [4]:
results = []
for model_name, model_summary in summary_misdemeanor6_ky_model.items():
    results.append([model_name, 
                    np.mean(model_summary['KY_validation']), 
                    np.mean(model_summary['auc_diff']), 
                    np.mean(model_summary['FL_score'])])
results

[['Logistic', 0.734957979697535, 0.001671533930320468, 0.5578144517022725],
 ['Lasso', 0.7353044754530371, 0.001260739818244727, 0.5584709998686426],
 ['LinearSVM', 0.7382006219068021, 0.001557557031716783, 0.5499659664919214],
 ['RF', 0.7597303600383061, 0.02881160802870477, 0.5523466402359657],
 ['XGBoost', 0.76600330274434, 0.027126818397479324, 0.5633235810415448]]

In [5]:
path = "./results/baselines/six-month/"
results = [["Misdemeanor",             
            np.str(round(np.mean(logistic_summary['FL_score']),3)) + " (" + np.str(round(np.std(logistic_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(lasso_summary['FL_score']), 3)) + " (" + np.str(round(np.std(lasso_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(svm_summary['FL_score']),3)) + " (" + np.str(round(np.std(svm_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(rf_summary['FL_score']),3)) + " (" + np.str(round(np.std(rf_summary['FL_score']),3)) + ")", 
            np.str(round(np.mean(xgb_summary['FL_score']),3)) + " (" + np.str(round(np.std(xgb_summary['FL_score']),3)) + ")"]]
with open(path + 'six-month-ky-baseline-fl-score.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

In [6]:
path = "./results/baselines/six-month/"
results = [["Misdemeanor",             
            np.str(round(np.mean(logistic_summary['KY_score']),3)) + " (" + np.str(round(np.std(logistic_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(lasso_summary['KY_score']), 3)) + " (" + np.str(round(np.std(lasso_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(svm_summary['KY_score']),3)) + " (" + np.str(round(np.std(svm_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(rf_summary['KY_score']),3)) + " (" + np.str(round(np.std(rf_summary['KY_score']),3)) + ")", 
            np.str(round(np.mean(xgb_summary['KY_score']),3)) + " (" + np.str(round(np.std(xgb_summary['KY_score']),3)) + ")"]]
with open(path + 'six-month-ky-baseline-ky-score.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)